# --- Day 7: Recursive Circus ---

Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:

```
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
```

...then you would be able to recreate the structure of the towers that looks like this:

```
                gyxo
              /     
         ugml - ebii
       /      \     
      |         jptl
      |        
      |         pbga
     /        /
tknk --- padx - havc
     \        \
      |         qoyq
      |             
      |         ktlj
       \      /     
         fwft - cntj
              \     
                xhth
```

In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

## a non graph hacky way to answe this

The obvious way to answer this is to build a graph, but since the problem is so well defined - there is only one parent node, we can figure this out without lists.

First, the test input:

In [2]:
test_case = """pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)""".split("\n")

print(f"the test case answer is: tknk")
test_case

the test case answer is: tknk


['pbga (66)',
 'xhth (57)',
 'ebii (61)',
 'havc (66)',
 'ktlj (57)',
 'fwft (72) -> ktlj, cntj, xhth',
 'qoyq (66)',
 'padx (45) -> pbga, havc, qoyq',
 'tknk (41) -> ugml, padx, fwft',
 'jptl (61)',
 'ugml (68) -> gyxo, ebii, jptl',
 'gyxo (61)',
 'cntj (57)']

Now a list of all the nodes:

In [3]:
nodes = [line.split()[0] for line in test_case]
nodes[:5]

['pbga', 'xhth', 'ebii', 'havc', 'ktlj']

Now a list of just the parent nodes (the ones which have children):

In [4]:
parents = [line.split()[0] for line in test_case if "->" in line]
parents

['fwft', 'padx', 'tknk', 'ugml']

In [5]:
children_nodes = [line.split()[3:] for line in test_case if "->" in line]
children_nodes

[['ktlj,', 'cntj,', 'xhth'],
 ['pbga,', 'havc,', 'qoyq'],
 ['ugml,', 'padx,', 'fwft'],
 ['gyxo,', 'ebii,', 'jptl']]

I want a nice flat list of all the children nodes, so I use chain to flatten out the list, and also here I strip the punctuation characters from the nodes:

In [7]:
import re
from itertools import chain
childs = [re.sub('[\W_]+', "", node) for node in chain(*children_nodes)]
childs

['ktlj',
 'cntj',
 'xhth',
 'pbga',
 'havc',
 'qoyq',
 'ugml',
 'padx',
 'fwft',
 'gyxo',
 'ebii',
 'jptl']

the answer should just be:

In [8]:
[node for node in parents if node not in childs]

['tknk']

Now to make it into a function:

In [9]:
from itertools import chain
    
def top_node(test_case):
    """takes in a tree in text form and returns the name of the parent node"""
    parents = [line.split()[0] for line in test_case if "->" in line]
    children_nodes = [line.split()[3:] for line in test_case if "->" in line]
    childs = [re.sub('[\W_]+', "", node) for node in chain(*children_nodes)]
    return [node for node in parents if node not in childs]

top_node(test_case)

['tknk']

Now to read in the puzzle data and get the parent node:

In [10]:
with open('puzzle_inputs/day7_input.txt') as f:
    data = f.read().split("\n")
data = [line for line in data if len(line)>0]
data[:5]

['tqefb (40)',
 'lhrml (164) -> ecblhee, sdjshz',
 'ykntwjm (16)',
 'fbebcq (233) -> ilzfg, vqbvnf, idyiyg, tifpswp',
 'rqjpza (1043) -> xszbzi, zafhcbb, qoouyiw']

In [11]:
top_node(data)

['cyrupz']

And that works! But, to do part two, I need to rewrite this...

# --- Part Two ---

The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.

In the example above, this means that for ugml's disc to be balanced, gyxo, ebii, and jptl must all have the same weight, and they do: 61.

However, for tknk to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same:

```
ugml + (gyxo + ebii + jptl) = 68 + (61 + 61 + 61) = 251
padx + (pbga + havc + qoyq) = 45 + (66 + 66 + 66) = 243
fwft + (ktlj + cntj + xhth) = 72 + (57 + 57 + 57) = 243
```

As you can see, tknk's disc is unbalanced: ugml's stack is heavier than the other two. Even though the nodes above ugml are balanced, ugml itself is too heavy: it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced. If this change were made, its weight would be 60.

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

I'm going to build a tower:

In [41]:
class Tower:
    def __init__(self, name, weight, parent=None, sub_towers=[]):
        self.name = name
        self.weight = weight
        self.parent = parent
        self.sub_towers = sub_towers
    
    def total_weight():
        """returns total weight of the tower"""
        if self.sub_towers:
            sub_towers_weight = sum([t.total_weight() for t in self.sub_towers])
            print(sub_towers_weight)
            return self.weight + sub_towers_weight
        else:
            return self.weight
        
    def __str__(self):
        return f"{self.name}, {self.weight}, has {sub_towers} subtowers"

In [42]:
def build_tower(test_case):
    """takes in a list of strings representing the nodes, weights and their sub nodes, returns a tower"""
    return False

In [44]:
def parse_line(tower_desc):
    tower = tower_desc.split()
    name = tower[0]
    weight = re.sub('[\W_]+', "", tower[1])
    sub_towers = []
    
    # deal with subtowers
    if len(line) > 3:
        sub_towers = [re.sub('[\W]+', "", node) for node in line[3:]]
        
    return name, weight, sub_towers
    
for line in test_case:
    line = line.split()
    name = line[0]
    weight = re.sub('[\W_]+', "", line[1])
    sub_towers = []
    
    # deal with subtowers
    if len(line) > 3:
        sub_towers = [re.sub('[\W]+', "", node) for node in line[3:]]
    
    t = Tower(name, weight, None, sub_towers)
    print(t)

pbga, 66, has [] subtowers
xhth, 57, has [] subtowers
ebii, 61, has [] subtowers
havc, 66, has [] subtowers
ktlj, 57, has [] subtowers
fwft, 72, has ['ktlj', 'cntj', 'xhth'] subtowers
qoyq, 66, has [] subtowers
padx, 45, has ['pbga', 'havc', 'qoyq'] subtowers
tknk, 41, has ['ugml', 'padx', 'fwft'] subtowers
jptl, 61, has [] subtowers
ugml, 68, has ['gyxo', 'ebii', 'jptl'] subtowers
gyxo, 61, has [] subtowers
cntj, 57, has [] subtowers
